In [ ]:

import subprocess
import grpc
from mlops.v2_dataplane.v2_dataplane_pb2_grpc import GRPCInferenceServiceStub
from mlops.v2_dataplane.v2_dataplane_pb2 import ModelInferRequest, InferTensorContents

def get_mesh_ip():
    cmd = f"kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{{.status.loadBalancer.ingress[0].ip}}'"
    return subprocess.check_output(cmd, shell=True).decode("utf-8")

def generate_requests():
    prompts = ["Once upon a time " * 10_000]
    for prompt in prompts:
        yield ModelInferRequest(
            model_name="text-model",
            inputs=[ModelInferRequest.InferInputTensor(
                name="prompt",
                datatype="BYTES",
                shape=[1, 1],
                contents=InferTensorContents(bytes_contents=[prompt.encode()])
            )]
        )


endpoint = get_mesh_ip() + ":80"
channel = grpc.insecure_channel(endpoint)
stub = GRPCInferenceServiceStub(channel)


# Define headers (metadata)
metadata = [
    ("seldon-model", "text-model"),
    ("seldon-internal-model", "text-model_1")
]

# Make the streaming request with metadata
response_stream = stub.ModelStreamInfer(generate_requests(), metadata=metadata)

for response in response_stream:
    print(response)